In [3]:
# Import dependencies
import pandas as pd
import json
import re

In [4]:
# Import data
missing_activities_path = "../Resources/missingActivities.csv"
missing_df = pd.read_csv(missing_activities_path)

In [5]:
# Clean up data to only include what is needed
missing_df = missing_df[["latitude","longitude","City","State/Province","Country"]]
missing_df = missing_df.rename(columns={"City":"city","State/Province":"state/province","Country":"country"})
missing_df['city'] = missing_df['city'].str.replace('\u200b','')
missing_df['country'] = missing_df['country'].str.replace('\u200b','')

In [6]:
# Convert the locations to a list
cities = missing_df['city'].tolist()
countries = missing_df['country'].tolist()
locations = list(zip(cities,countries))

In [7]:
# Clean up the list for adding to ChatGPT
for i in range(len(locations)):
    if locations[i][0] == locations[i][1]:
        locations[i] = locations[i][0]
locations.remove(("Marrakech","Morocco"))

In [8]:
# Categories from original YELP API Call
categories = ['beaches','hanggliding','horsebackriding',
             'hiking','hot_air_balloons',
              'paddleboarding','parasailing',
              'sailing','snorkeling','ziplining', 
              'spas','hotsprings','massage','bedbreakfast','hotels','resorts',
              'skiresorts','tours', 'transportation','restaurants','nightlife'
             ]

In [9]:
# Separate out non-activities for later use
non_activities = ['bedbreakfast','hotels','resorts','skiresorts','transportation','restaurants','nightlife']

## Pull Activities Lists From Chat GPT

In [10]:
# Create list of only activities
activities = [i for i in categories if i not in non_activities]

In [11]:
# Split list of locations into sets of 20 for ChatGPT searches
locations_1 = locations[:20]
locations_2 = locations[20:40]
locations_3 = locations[40:60]
locations_4 = locations[60:]
print(len(locations_1),len(locations_2),len(locations_3),len(locations_4))

20 20 20 13


In [12]:
# Create search criteria
search1 = "Provide the top 5 activities for each of the following locations in table format:" + str(locations_1) + ". Only use the following words for the activities:" + str(activities)
search2 = "Provide the top 5 activities for each of the following locations in table format:" + str(locations_2) + ". Only use the following words for the activities:" + str(activities)
search3 = "Provide the top 5 activities for each of the following locations in table format:" + str(locations_3) + ". Only use the following words for the activities:" + str(activities)
search4 = "Provide the top 5 activities for each of the following locations in table format:" + str(locations_4) + ". Only use the following words for the activities:" + str(activities)

In [13]:
search1 #Move the results to ChatGPT

"Provide the top 5 activities for each of the following locations in table format:['Cook Islands', ('Dubrovnik', 'Croatia'), 'Antigua and Barbuda', ('Dubai', 'United Arab Emirates'), 'Turks and Caicos Islands', ('Stellenbosch', ' South Africa'), ('Zanzibar', 'Tanzania'), 'Mauritius', 'Cayman Islands', 'Seychelles', ('Marrakesh', 'Morocco'), ('Jaipur', ' India'), 'Barbados', ('Udaipur', ' India'), 'Jamaica', ('Hermanus', ' South Africa'), ('Agra', ' India'), 'Fiji', ('St. Petersburg', 'Russia'), 'Saint Lucia']. Only use the following words for the activities:['beaches', 'hanggliding', 'horsebackriding', 'hiking', 'hot_air_balloons', 'paddleboarding', 'parasailing', 'sailing', 'snorkeling', 'ziplining', 'spas', 'hotsprings', 'massage', 'tours']"

In [14]:
search2 #Move the results to ChatGPT

"Provide the top 5 activities for each of the following locations in table format:['Bahamas', ('Bali', 'Indonesia'), ('Bangkok', 'Thailand'), ('New Delhi', 'India'), ('Budapest', 'Hungary'), ('Mumbai', 'India'), ('Bucharest', 'Romania'), ('Phuket', 'Thailand'), ('Chiang Mai', 'Thailand'), ('Jakarta', 'Indonesia'), ('Sofia', 'Bulgaria'), ('Hanoi', 'Vietnam'), ('Ho Chi Minh City', 'Vietnam'), ('Belgrade', 'Serbia'), ('Siem Reap', 'Cambodia'), ('Kyiv', 'Ukraine'), ('Moscow', 'Russia'), ('Hoi An', 'Vietnam'), ('Colombo', 'Sri Lanka'), ('Seoul', 'South Korea')]. Only use the following words for the activities:['beaches', 'hanggliding', 'horsebackriding', 'hiking', 'hot_air_balloons', 'paddleboarding', 'parasailing', 'sailing', 'snorkeling', 'ziplining', 'spas', 'hotsprings', 'massage', 'tours']"

In [15]:
search3.replace("\"","")  #Move the results to ChatGPT

"Provide the top 5 activities for each of the following locations in table format:[('Medellin', 'Colombia'), ('Kathmandu', 'Nepal'), ('Johannesburg', 'South Africa'), ('Phnom Penh', 'Cambodia'), ('Beijing', 'China'), ('Lima', 'Peru'), ('Minsk', 'Belarus'), ('Cartagena', 'Colombia'), ('Ljubljana', 'Slovenia'), ('Quito', 'Ecuador'), ('Tallinn', 'Estonia'), ('Zagreb', 'Croatia'), (Xi'an, 'China'), ('Arequipa', 'Peru'), ('Montevideo', 'Uruguay'), ('Split', 'Croatia'), ('Pokhara', 'Nepal'), ('Shanghai', 'China'), ('Riga', 'Latvia'), ('Vilnius', 'Lithuania')]. Only use the following words for the activities:['beaches', 'hanggliding', 'horsebackriding', 'hiking', 'hot_air_balloons', 'paddleboarding', 'parasailing', 'sailing', 'snorkeling', 'ziplining', 'spas', 'hotsprings', 'massage', 'tours']"

In [16]:
search4

"Provide the top 5 activities for each of the following locations in table format:[('Hvar', 'Croatia'), ('San Jose', 'Costa Rica'), ('Abu Dhabi', 'United Arab Emirates'), ('Vientiane', 'Laos'), ('La Paz', 'Bolivia'), ('Doha', 'Qatar'), ('Tel Aviv', 'Israel'), ('Dakar', 'Senegal'), ('Muscat', 'Oman'), ('Kuwait City', 'Kuwait'), ('Beirut', 'Lebanon'), ('Valletta', 'Malta'), ('Reykjavik', 'Iceland')]. Only use the following words for the activities:['beaches', 'hanggliding', 'horsebackriding', 'hiking', 'hot_air_balloons', 'paddleboarding', 'parasailing', 'sailing', 'snorkeling', 'ziplining', 'spas', 'hotsprings', 'massage', 'tours']"

In [17]:
# Import ChatGPT results
json_path = "../Resources/chat_gpt_results.json"
chat_data = json.load(open(json_path))

In [18]:
# Create a list based on the portion of the code we need. Skip where the result doesn't have the needed part
results = []
for i in chat_data[0]['mapping'].keys():
    try:
        result = chat_data[0]['mapping'][i]['message']['content']['parts']
        results.append(result)
    except:
        pass
# Merge the lists into a single list based on the results we want
list_results = results[2]+results[4]+results[6]+results[8]

In [19]:
# Normalize the list by removing all unnecessary characters
for i in range(len(list_results)):
    list_results[i] = list_results[i].replace('\n','')
    list_results[i] = list_results[i].replace('--','')
    list_results[i] = list_results[i].replace('|-|-|','')
    list_results[i] = list_results[i].replace ('  ','')
    list_results[i] = list_results[i].replace ('| ','|')
    list_results[i] = list_results[i].replace (' |','|')
    list_results[i] = list_results[i].split('||')
    # Pop off the last pipe from the list
    list_results[i][-1] = list_results[i][-1][:-1] 
    
# Merge the list of lists into a single list
one_list = [item for sublist in list_results for item in sublist]

In [20]:
# Remove the header rows to standardize data
to_remove = ['|Location|Top 5 Activities','Here are the top 5 activities for each of the following locations in table format:|Location|Activities']
one_list = [i for i in one_list if i not in to_remove]
one_list

['Cook Islands|beaches, snorkeling, paddleboarding, sailing, spas',
 'Dubrovnik, Croatia|hiking, sailing, tours, beaches, spas',
 'Antigua and Barbuda|beaches, sailing, snorkeling, paddleboarding, horsebackriding',
 'Dubai, UAE|hot_air_balloons, tours, spas, horsebackriding, sailing',
 'Turks and Caicos Islands|beaches, snorkeling, parasailing, paddleboarding, horsebackriding',
 'Stellenbosch, South Africa|hiking, horsebackriding, wine tours, spas, hot springs',
 'Zanzibar, Tanzania|snorkeling, beaches, tours, sailing, spice tours',
 'Mauritius|beaches, snorkeling, spas, hiking, sailing',
 'Cayman Islands|beaches, snorkeling, sailing, paddleboarding, spas',
 'Seychelles|beaches, snorkeling, sailing, hiking, spas',
 'Marrakesh, Morocco|hiking, hot_air_balloons, tours, spas, ziplining',
 'Jaipur, India|tours, hot_air_balloons, horsebackriding, spas, hiking',
 'Barbados|beaches, snorkeling, sailing, horsebackriding, spas',
 'Udaipur, India|tours, hot_air_balloons, hiking, horsebackriding,

In [21]:
# Convert list to dataframe
results_df = pd.DataFrame(one_list)
# Rename Column
results_df = results_df.rename(columns={0:"location"})
# Split column into two
results_df[['location','Activities']] = results_df['location'].str.split('|',expand=True)
# Split all activities into separate columns
results_df[['act1','act2','act3','act4','act5']] = results_df["Activities"].str.split(',',expand=True)

In [22]:
# Fix all locations without cities to normalize back to dataset
for i in range(results_df.shape[0]):
    string = results_df.loc[i].at['location']
    if len(string.split(',')) != 2:
        string = string + ", " + string
        results_df.loc[i].at['location'] = string
        
# Split location column into two        
results_df[["city","country"]] = results_df["location"].str.split(',',expand=True)
results_df = results_df.drop(["location","Activities"],axis=1)
# Rearrange columns

In [23]:
results_df

,act1,act2,act3,act4,act5,city,country
0,beaches,snorkeling,paddleboarding,sailing,spas,Cook Islands,Cook Islands
1,hiking,sailing,tours,beaches,spas,Dubrovnik,Croatia
2,beaches,sailing,snorkeling,paddleboarding,horsebackriding,Antigua and Barbuda,Antigua and Barbuda
3,hot_air_balloons,tours,spas,horsebackriding,sailing,Dubai,UAE
4,beaches,snorkeling,parasailing,paddleboarding,horsebackriding,Turks and Caicos Islands,Turks and Caicos Islands
...,...,...,...,...,...,...,...
68,Beaches,hot air balloons,snorkeling,tours,sailing,Muscat,Oman
69,Beaches,tours,spas,hot springs,horsebackriding,Kuwait City,Kuwait
70,Beaches,tours,spas,hiking,hot springs,Beirut,Lebanon
71,Beaches,snorkeling,sailing,tours,spas,Valletta,Malta


In [24]:
results_df.to_csv("../Resources/chatgpt_activities.csv")